# CHRONOS Demo Walkthrough

> Conditional Hierarchical Regime-Optimized Navigation & Oversight System

This notebook demonstrates the full CHRONOS pipeline from data loading to portfolio optimization.

## 1. Setup and Configuration

In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import warnings

warnings.filterwarnings('ignore')

# Add project root to path
sys.path.insert(0, os.path.dirname(os.getcwd()))

# Import CHRONOS modules
import config
from config import (
    PORTFOLIO_ASSETS, REGIME_CONSTRAINTS, INITIAL_CAPITAL,
    TRAIN_START, TRAIN_END, TEST_START, TEST_END
)

print('✅ CHRONOS modules loaded successfully')
print(f'📊 Portfolio Assets: {PORTFOLIO_ASSETS}')
print(f'💰 Initial Capital: ${INITIAL_CAPITAL:,}')
print(f'📅 Training: {TRAIN_START} to {TRAIN_END}')
print(f'📅 Testing: {TEST_START} to {TEST_END}')

## 2. Data Pipeline

Load market data and verify no missing values.

In [ ]:
from src.data.data_loader import DataLoader

# Initialize data loader
loader = DataLoader()

# Download data
raw_data = loader.get_data(
    tickers=['^GSPC', '^VIX', 'SPY', 'TLT', 'GLD'],
    start_date=config.DATA_START_DATE,
    end_date=config.DATA_END_DATE
)

print(f'Downloaded data shape: {raw_data.shape}')
print(f'Date range: {raw_data.index.min()} to {raw_data.index.max()}')
print(f'Missing values: {raw_data.isnull().sum().sum()}')

# Display sample
raw_data.head()

## 3. Feature Engineering

Generate technical indicators with `.shift(1)` to prevent lookahead bias.

In [ ]:
from src.data.feature_engineering import FeatureEngineer

# Initialize feature engineer
engineer = FeatureEngineer(config)

# Generate features
features = engineer.engineer_features(raw_data)

print(f'Generated {features.shape[1]} features')
print(f'Feature columns: {list(features.columns)[:10]}...')

# Verify .shift(1) was applied
print('\n✅ Anti-Leakage Validation:')
print(f'First row has NaN (due to shift): {features.iloc[0].isnull().any()}')

features.head()

## 4. Regime Detection with HMM

Train a Gaussian Hidden Markov Model to detect market regimes.

In [ ]:
import pickle
from hmmlearn import hmm

# Try to load pre-trained model
model_path = os.path.join(config.MODELS_DIR, 'regime_detector.pkl')

if os.path.exists(model_path):
    with open(model_path, 'rb') as f:
        regime_detector = pickle.load(f)
    print('✅ Loaded pre-trained regime detector')
else:
    print('Training new regime detector...')
    
    # Prepare features for HMM
    hmm_features = features[['log_return_gspc', 'volatility_20d', 'vix_level']].dropna()
    
    # Train HMM
    regime_detector = hmm.GaussianHMM(
        n_components=3,
        covariance_type='full',
        n_iter=1000,
        random_state=42
    )
    regime_detector.fit(hmm_features.values)
    print('✅ Trained regime detector')

print(f'Number of states: {regime_detector.n_components}')

In [ ]:
# Predict regimes
hmm_features = features[['log_return_gspc', 'volatility_20d', 'vix_level']].dropna()
regimes = regime_detector.predict(hmm_features.values)
regime_probs = regime_detector.predict_proba(hmm_features.values)
confidence = regime_probs.max(axis=1)

# Create regime DataFrame
regime_df = pd.DataFrame({
    'regime': regimes,
    'confidence': confidence
}, index=hmm_features.index)

# Regime distribution
print('Regime Distribution:')
regime_counts = pd.Series(regimes).value_counts().sort_index()
regime_names = {0: 'Euphoria', 1: 'Complacency', 2: 'Capitulation'}
for r, count in regime_counts.items():
    pct = count / len(regimes) * 100
    print(f'  {regime_names.get(r, r)}: {pct:.1f}%')

In [ ]:
# Visualize regimes
fig, ax = plt.subplots(figsize=(14, 6))

# Get price data
if 'close_gspc' in raw_data.columns:
    prices = raw_data['close_gspc'].loc[hmm_features.index]
else:
    prices = raw_data['Close'].loc[hmm_features.index]

# Plot price
ax.plot(prices.index, prices.values, color='black', linewidth=1.5, label='S&P 500')

# Color backgrounds by regime
regime_colors = {0: '#00C853', 1: '#FFD600', 2: '#D50000'}
for i in range(1, len(regimes)):
    ax.axvspan(
        hmm_features.index[i-1], hmm_features.index[i],
        color=regime_colors[regimes[i]], alpha=0.3
    )

ax.set_title('CHRONOS Regime Detection', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Price')
ax.legend()

# Add legend for regimes
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#00C853', alpha=0.3, label='Euphoria'),
    Patch(facecolor='#FFD600', alpha=0.3, label='Complacency'),
    Patch(facecolor='#D50000', alpha=0.3, label='Capitulation')
]
ax.legend(handles=legend_elements, loc='upper left')

plt.tight_layout()
plt.show()

## 5. Transition Matrix

In [ ]:
# Display transition matrix
transition_matrix = regime_detector.transmat_

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(
    transition_matrix,
    annot=True,
    fmt='.2f',
    cmap='Blues',
    xticklabels=['Euphoria', 'Complacency', 'Capitulation'],
    yticklabels=['Euphoria', 'Complacency', 'Capitulation'],
    ax=ax
)
ax.set_title('Regime Transition Matrix', fontsize=14, fontweight='bold')
ax.set_xlabel('To Regime')
ax.set_ylabel('From Regime')
plt.tight_layout()
plt.show()

print('\nTransition Matrix Interpretation:')
print('- Diagonal values show regime persistence')
print('- Off-diagonal show transition probabilities')

## 6. Portfolio Allocation by Regime

Display regime-specific allocation constraints.

In [ ]:
# Display regime constraints
print('Regime-Specific Portfolio Constraints:\n')

for regime_id, name in regime_names.items():
    constraints = REGIME_CONSTRAINTS[regime_id]
    print(f'{name} (Regime {regime_id}):')
    for asset, (min_wt, max_wt) in constraints.items():
        mid = (min_wt + max_wt) / 2
        print(f'  {asset}: {min_wt:.0%} - {max_wt:.0%} (target: {mid:.0%})')
    print()

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

for idx, (regime_id, name) in enumerate(regime_names.items()):
    constraints = REGIME_CONSTRAINTS[regime_id]
    allocations = [(constraints[a][0] + constraints[a][1]) / 2 for a in PORTFOLIO_ASSETS]
    
    wedges, texts, autotexts = axes[idx].pie(
        allocations,
        labels=PORTFOLIO_ASSETS,
        autopct='%1.0f%%',
        colors=['#1f77b4', '#2ca02c', '#ff7f0e']
    )
    axes[idx].set_title(name)

plt.suptitle('Target Allocations by Regime', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Performance Summary

Load backtest results if available.

In [ ]:
import json

# Try to load backtest results
results_path = os.path.join(config.OUTPUTS_DIR, 'backtest', 'summary_statistics.json')

if os.path.exists(results_path):
    with open(results_path, 'r') as f:
        summary_stats = json.load(f)
    
    print('CHRONOS Performance Summary')
    print('=' * 40)
    
    metrics = [
        ('Total Return', 'total_return', '%'),
        ('Sharpe Ratio', 'sharpe_ratio', ''),
        ('Sortino Ratio', 'sortino_ratio', ''),
        ('Max Drawdown', 'max_drawdown', '%'),
        ('Calmar Ratio', 'calmar_ratio', ''),
        ('Win Rate', 'win_rate', '%'),
        ('Volatility', 'volatility', '%')
    ]
    
    for label, key, suffix in metrics:
        if key in summary_stats:
            val = summary_stats[key]
            if suffix == '%':
                print(f'{label}: {val:.2%}')
            else:
                print(f'{label}: {val:.2f}')
else:
    print('⚠️ No backtest results found. Run the backtest first:')
    print('   python -m src.backtest.run_backtest')

## 8. Current Status

Display the latest regime and recommended allocation.

In [ ]:
# Get latest regime
latest_regime = regimes[-1]
latest_confidence = confidence[-1]
latest_date = hmm_features.index[-1]

print('CHRONOS Current Status')
print('=' * 40)
print(f'Date: {latest_date.strftime("%Y-%m-%d")}')
print(f'Regime: {regime_names[latest_regime]}')
print(f'Confidence: {latest_confidence:.1%}')

# Get recommended allocation
constraints = REGIME_CONSTRAINTS[latest_regime]
print('\nRecommended Allocation:')
for asset in PORTFOLIO_ASSETS:
    min_wt, max_wt = constraints[asset]
    target = (min_wt + max_wt) / 2
    print(f'  {asset}: {target:.0%}')

# Status emoji
status_emoji = {0: '🟢', 1: '🟡', 2: '🔴'}
print(f'\nStatus: {status_emoji[latest_regime]} {regime_names[latest_regime]}')

## 9. Next Steps

To run the full CHRONOS system:

1. **Train Models**: `python -m src.models.train_regime_detector`
2. **Run Backtest**: `python -m src.backtest.run_backtest`
3. **Launch Dashboard**: `streamlit run app.py`

For more information, see:
- [docs/ARCHITECTURE.md](../docs/ARCHITECTURE.md)
- [docs/METHODOLOGY.md](../docs/METHODOLOGY.md)
- [docs/DASHBOARD_GUIDE.md](../docs/DASHBOARD_GUIDE.md)